# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122712e+02     1.551663e+00
 * time: 0.07651495933532715
     1     1.141622e+01     8.747565e-01
 * time: 1.4261839389801025
     2    -1.264994e+01     9.946860e-01
 * time: 1.5238399505615234
     3    -3.429007e+01     7.698666e-01
 * time: 1.6551969051361084
     4    -4.834649e+01     5.461153e-01
 * time: 1.8061888217926025
     5    -5.718844e+01     2.093628e-01
 * time: 1.9233059883117676
     6    -5.988488e+01     1.048598e-01
 * time: 2.014204978942871
     7    -6.091353e+01     5.451258e-02
 * time: 2.1052498817443848
     8    -6.131493e+01     7.064753e-02
 * time: 2.1964609622955322
     9    -6.161448e+01     3.424745e-02
 * time: 2.3011088371276855
    10    -6.181200e+01     2.360874e-02
 * time: 2.3921549320220947
    11    -6.196857e+01     2.015299e-02
 * time: 2.483119010925293
    12    -6.203475e+01     1.969486e-02
 * time: 2.574676990509033
    13    -6.210696e+01     1.668801e-02
 * time: 2.665302991867

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671057
    AtomicLocal         -18.8557552
    AtomicNonlocal      14.8522551
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485340 
    Xc                  -19.3336809

    total               -62.261666451506
